# FreshDesk API

In [16]:
import pandas as pd
import requests
import json
import numpy as np

In [17]:
API_KEY = "API_KEY"
MyDomain = "Domain" #e.g. for testdomain.freshworks.com, the MyDomain = testdomain
password = "password"

## Tickets & Emails Request

In [29]:
tickets=[]
emailMessage =[]

responseAllTickets = requests.get("https://"+ MyDomain +".freshdesk.com/api/v2/tickets", auth = (API_KEY, password)).json()
for ticket in responseAllTickets:
        tickets.append(ticket)
        
        tempId = str(ticket['id'])
        ticketConversation = requests.get("https://"+ MyDomain +".freshdesk.com/api/v2/tickets/"+tempId+"/conversations", auth = (API_KEY, password)).json()
        if str(ticketConversation) !='[]':
            for email in ticketConversation:
                emailMessage.append(email)
        
Tickets = pd.DataFrame(tickets)
emailHistory = pd.DataFrame(emailMessage)

## Internal Comms Table

In [19]:
#Below includes notes and internal emails

internalEmailHistory = emailHistory[emailHistory.private==True]
internalEmailHistory = internalEmailHistory[['body_text','id','ticket_id','user_id','to_emails','from_email',
                    'attachments','created_at','updated_at','email_failure_count','outgoing_failures']].rename(columns={'id':'ID',
                    'body_text':'BodyText','ticket_id':'TicketID','user_id':'UserID','to_emails':'ToEmails',
                    'from_email':'FromEmail','created_at':'CreatedDt','updated_at':'UpdatedDt',
                    'email_failure_count':'EmailFailureCount','outgoing_failures':'OutgoingFailures'})
internalEmailHistory['Attachments'] = np.where(internalEmailHistory['attachments'].str.len()==0, False, True)
internalEmailHistory['CreatedDt'] = pd.to_datetime(internalEmailHistory['CreatedDt']).dt.date
internalEmailHistory['UpdatedDt'] = pd.to_datetime(internalEmailHistory['UpdatedDt']).dt.date

In [20]:
#Internal Type based on Body Text of communication

internalEmailsConditions = [internalEmailHistory['BodyText'].str.contains("Abandoned call from"),
                           internalEmailHistory['BodyText'].str.contains("Voicemail from"),
                           internalEmailHistory['BodyText'].str.contains("This ticket is closed and merged into ticket"),
                           internalEmailHistory['BodyText'].str.contains("Unanswered call"),
                           internalEmailHistory['BodyText'].str.contains("Conversation between")]
internalEmailsResults = ["Abandoned Call","Voicemail","Merged Ticket","Unanswered Call","Internal Conversation"] 

internalEmailHistory['InternalType'] = np.select(internalEmailsConditions,internalEmailsResults, default = "Notes")

## External Comms Table

In [21]:
emailHistory = emailHistory[emailHistory.private==False]
emailHistory = emailHistory[['id','incoming','ticket_id','user_id','to_emails','from_email','body_text',
              'attachments','created_at','updated_at','email_failure_count','outgoing_failures']].rename(columns={"id":"ID",
              "ticket_id":"TicketID","user_id":"UserID","category":"Category","to_emails":"ToEmails","from_email":"FromEmail",
              "created_at":"CreatedDt", "updated_at":"UpdatedDt",'email_failure_count':'EmailFailureCount',
              'outgoing_failures':'OutgoingFailures'})
emailHistory['Attachments'] = np.where(emailHistory['attachments'].str.len()==0, False, True)
emailHistory['CreatedDt'] = pd.to_datetime(emailHistory['CreatedDt']).dt.date
emailHistory['UpdatedDt'] = pd.to_datetime(emailHistory['UpdatedDt']).dt.date

## Tickets Table

In [22]:
Tickets = Tickets.astype({'company_id':'str', 'requester_id':'str','responder_id':'str','id':'str','group_id':'str',
                          'source':'str','priority':'str','status':'str'})
Tickets['company_id'] = Tickets['company_id'].str[:-2]
Tickets['responder_id'] = Tickets['responder_id'].str[:-2]
Tickets['group_id'] = Tickets['group_id'].str[:-2]
Tickets['fr_due_by'] = pd.to_datetime(Tickets['fr_due_by']).dt.date
Tickets['created_at'] = pd.to_datetime(Tickets['created_at']).dt.date
Tickets['updated_at'] = pd.to_datetime(Tickets['updated_at']).dt.date

In [23]:
Tickets = Tickets[['id','priority','group_id','type','source','company_id','status','requester_id', 'responder_id',
                'fr_due_by','fr_escalated','product_id','is_escalated','associated_tickets_count','created_at', 
                'updated_at']].rename(columns={'id':'ID','priority':'Priority','group_id':'GroupID','type':'Type',
                'source':'Source','company_id':'CompanyID','status':'Status','requester_id':'RequesterID',
                'responder_id':'ResponderID','fr_escalated':'FirstResponseEscalated','product_id':'ProductID',
                'is_escalated':'IsEscalated','associated_tickets_count':'AssociatedTicketsCount',
                'fr_due_by':'FirstResponseDueDt','created_at':'CreatedDt', 'updated_at':'UpdatedDt'})

In [24]:
def ticketSource(source):
    switcher={
        "1":"Email",
        "2":"Portal",
        "3":"Phone",
        "4":"Forum",
        "5":"Twitter",
        "6":"Facebook",
        "7":"Chat",
        "8":"Mobihelp",
        "9":"Feedback Widget",
        "10":"Outbound Email",
        "11":"Ecommerce"
        }
    return switcher.get(source,"UnknownSource")

Tickets['Source'] =Tickets['Source'].apply(ticketSource)

In [25]:
def ticketPriority(priority):
    switcher={
        "1":"Low",
        "2":"Medium",
        "3":"High",
        "4":"Urgent"
        }
    return switcher.get(priority,"UnknownPriority")

Tickets['Priority'] =Tickets['Priority'].apply(ticketPriority)

In [26]:
def ticketStatus(status):
    switcher={
        "2":"Open",
        "3":"Pending",
        "4":"Resolved",
        "5":"Closed"
        }
    return switcher.get(status,"UnknownStatus")

Tickets['Status'] =Tickets['Status'].apply(ticketStatus)

## Groups Table

In [33]:
groups=[]
responseAllGroups = requests.get("https://"+ MyDomain +".freshdesk.com/api/v2/groups/", auth = (API_KEY, password)).json()
for group in responseAllGroups:
        groups.append(group)
        
        
Groups = pd.DataFrame(groups)

In [35]:
Groups['created_at'] = pd.to_datetime(Groups['created_at']).dt.date
Groups['updated_at'] = pd.to_datetime(Groups['updated_at']).dt.date

Groups = Groups[['id','name','created_at','updated_at']].rename(columns={"id":"ID","name":"Name","created_at":"CreatedDt",
                                                                         "updated_at":"UpdatedDt"})

## OpCos Table

In [40]:
companies=[]
responseAllCompanies = requests.get("https://"+ MyDomain +".freshdesk.com/api/v2/companies", auth = (API_KEY, password)).json()
for company in responseAllCompanies:
        companies.append(company)
        
        
Company = pd.DataFrame(companies)

## Contacts Table

In [45]:
contacts=[]
responseAllContacts = requests.get("https://"+ MyDomain +".freshdesk.com/api/v2/contacts?per_page=100", auth = (API_KEY, password)).json()
for contact in responseAllContacts:
        contacts.append(contact)

Contact = pd.DataFrame(contacts)

Contact = Contact[['id','active','email','job_title','mobile','name','phone','created_at']]